In [ ]:
from geofeather.pygeos import to_geofeather, from_geofeather
from osgeo import ogr
from tqdm import tqdm
from pygeos import from_wkb
import pandas

In [ ]:
# Functions from CISI https://github.com/snirandjan/CISI/

def query_b(geoType,keyCol,**valConstraint):
    """
    This function builds an SQL query from the values passed to the retrieve() function.
    Arguments:
         *geoType* : Type of geometry (osm layer) to search for.
         *keyCol* : A list of keys/columns that should be selected from the layer.
         ***valConstraint* : A dictionary of constraints for the values. e.g. WHERE 'value'>20 or 'value'='constraint'
    Returns:
        *string: : a SQL query string.
    """
    query = "SELECT " + "osm_id"
    for a in keyCol: query+= ","+ a  
    query += " FROM " + geoType + " WHERE "
    # If there are values in the dictionary, add constraint clauses
    if valConstraint: 
        for a in [*valConstraint]:
            # For each value of the key, add the constraint
            for b in valConstraint[a]: query += a + b
        query+= " AND "
    # Always ensures the first key/col provided is not Null.
    query+= ""+str(keyCol[0]) +" IS NOT NULL" 
    return query 

def retrieve(osm_path,geoType,keyCol,**valConstraint):
    """
    Function to extract specified geometry and keys/values from OpenStreetMap using pygeos
    Arguments:
        *osm_path* : file path to the .osm.pbf file of the region 
        for which we want to do the analysis.     
        *geoType* : Type of Geometry to retrieve. e.g. lines, multipolygons, etc.
        *keyCol* : These keys will be returned as columns in the dataframe.
        ***valConstraint: A dictionary specifiying the value constraints.  
        A key can have multiple values (as a list) for more than one constraint for key/value.  
    Returns:
        *Panda Core Series* : a frame with all columns, geometries as objects, and constraints specified.    
    """
    driver=ogr.GetDriverByName('OSM')
    data = driver.Open(osm_path)
    query = query_b(geoType,keyCol,**valConstraint)
    sql_lyr = data.ExecuteSQL(query)
    print(sql_lyr)
    features =[]
    # cl = columns 
    cl = ['osm_id'] 
    for a in keyCol: cl.append(a)
    if data is not None:
        print('query is finished, lets start the loop')
        for feature in tqdm(sql_lyr):
            print(feature.GetField(keyCol[0]))
            try:
                if feature.GetField(keyCol[0]) is not None:
                    geometry = feature.geometry()
                    if geometry is None:
                        print('geometry is non')
                        continue
                    wkb = geometry.ExportToWkb()
                    if not wkb:
                        print('wkb invalid')
                        continue
                    geom = from_wkb(wkb)
                    print(geom)
                    # geom = from_wkb(feature.geometry().ExportToWkb()) 
                    # if geom is None:
                    #     continue
                    # field will become a row in the dataframe.
                    field = []
                    print('Debug, field:', field)
                    for i in cl: field.append(feature.GetField(i))
                    field.append(geom)   
                    features.append(field)
            except: 
                print("WARNING: skipped OSM feature")   
                
    else:
        print("ERROR: Nonetype error when requesting SQL. Check required.")    
    cl.append('geometry')                   
    if len(features) > 0:
        return pandas.DataFrame(features,columns=cl)
    else:
        print("WARNING: No features or No Memory. returning empty GeoDataFrame") 
        return pandas.DataFrame(columns=['osm_id','geometry'])  

def roads_all(osm_path):
    """
    Function to extract road linestrings from OpenStreetMap  
    Arguments:
        *osm_path* : file path to the .osm.pbf file of the region 
        for which we want to do the analysis.        
    Returns:
        *GeoDataFrame* : a geopandas GeoDataFrame with all unique road linestrings.
    """   
    return (retrieve(osm_path,'lines',['highway'])).rename(columns={'highway': 'asset'}) 
    
def roads(osm_path):
    """
    Function to extract road linestrings categorized as primary, secondary and tertiary roads from OpenStreetMap  
    Arguments:
        *osm_path* : file path to the .osm.pbf file of the region 
        for which we want to do the analysis.        
    Returns:
        *GeoDataFrame* : a geopandas GeoDataFrame with all unique road linestrings.
    """   
    return (retrieve(osm_path,'lines',['highway'],**{'highway':["='motorway' or ","='motorway_link' or ","='trunk' or ","='trunk_link' or ","='primary' or ","='primary_link' or ","='secondary' or ","='secondary_link' or ","='tertiary' or ","='tertiary_link' or ","='residential' or ","='road' or ","='unclassified' or ","='living_street'"]})).rename(columns={'highway': 'asset'})  

def mainRoads(osm_path):
    """
    Function to extract main road linestrings from OpenStreetMap    
    Arguments:
        *osm_path* : file path to the .osm.pbf file of the region 
        for which we want to do the analysis.        
    Returns:
        *GeoDataFrame* : a geopandas GeoDataFrame with all unique main road linestrings.   
    """ 
    return retrieve(osm_path,'lines',['highway','oneway','lanes','maxspeed'],**{'highway':["='primary' or ","='trunk' or ","='motorway' or ","='motorway_link' or ","='trunk_link' or ","='primary_link' or ", "='secondary' or ","='tertiary' or ","='tertiary_link'"]})

In [ ]:
# Set OSM .pbf path
osm_path = r"D:/projects/sovereign-risk/Thailand/data/exposure/thailand-latest.osm.pbf"

In [ ]:
fetched_data_area = roads_all(osm_path) #extract required data

In [ ]:
fetched_data_area